In [6]:
import numpy as np
np.mean([1,2,None])

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [3]:
help(np.nanmean)

Help on function nanmean in module numpy.lib.nanfunctions:

nanmean(a, axis=None, dtype=None, out=None, keepdims=<no value>)
    Compute the arithmetic mean along the specified axis, ignoring NaNs.
    
    Returns the average of the array elements.  The average is taken over
    the flattened array by default, otherwise over the specified axis.
    `float64` intermediate and return values are used for integer inputs.
    
    For all-NaN slices, NaN is returned and a `RuntimeWarning` is raised.
    
    .. versionadded:: 1.8.0
    
    Parameters
    ----------
    a : array_like
        Array containing numbers whose mean is desired. If `a` is not an
        array, a conversion is attempted.
    axis : {int, tuple of int, None}, optional
        Axis or axes along which the means are computed. The default is to compute
        the mean of the flattened array.
    dtype : data-type, optional
        Type to use in computing the mean.  For integer inputs, the default
        is `float6

In [ ]:
def generate_data(n_samples, p):
    # p = [ Pr(y=1|x=0), Pr(y=1|x=1) ]
    x = [ np.random.choice(2) for i in range(n_samples) ]
    y = [ np.random.choice(2, p=[1-p[i], p[i]]) for i in x ]

    return x, y

class experiment:
    
    pre_drift_n = 500
    post_drift_n = 500
    
    def __init__(self, probs, detector):
        # probs = [a,b,c,d]
        # Before drift:
        # P(1|0) = a
        # P(1|1) = b
        # After drift:
        # P(1|0) = c
        # P(1|1) = d
        self.probs = probs
        self.detector = detector
        self.result = []
    def run_experiment(self):
        x1, y1 = genereate_data(experiment.pre_drift_n, self.probs[:2])
        x2, y2 = genereate_data(experiment.post_drift_n, self.probs[2:])
        # return either none or the point of detection
        self.results.append(result)
    def run_experiment(self, n_experiments):
        for i in range(n_experiments):
            self.run_experiment(n_experiments)
    def summary(self):
        detections = [i for i in self.results if i]
        return {'latency': np.mean(detections), 
                'detection_rate': len(detections) / len(self.results)}

# First Experiment: Low False Positive When Low Confidence

In [ ]:
experiment(0.6, 0.4, 0.6, 0.4)
